Author: Joshua <br />
Summary: Find travel time errors for not only the response time, but the serve time, hospital drive time and return time.

In [1]:
import csv
import collections
import pandas as pd
import numpy as np
import math
import json

## load the data

In [2]:
hosp = np.genfromtxt('Input_Data/calls_w_StnHospTimes.csv', delimiter=",", dtype=str)
f_wkc = np.genfromtxt('Input_Data/Full_WeekdayCalls.csv', delimiter=",", dtype=str)
wkc = np.genfromtxt('Input_Data/WeekdayCalls.csv', delimiter=",", dtype=str)
a_i = np.genfromtxt('Input_Data/austin_incidents.csv', delimiter=",", dtype=str)
#a_test_call = np.genfromtxt('Input_Data/austin_test_calls.csv', delimiter=",", dtype=str)

In [3]:
clean = np.genfromtxt('Input_Data/cleaned_data.csv', delimiter=",", dtype=str) #210k x 17
backend = np.genfromtxt('Input_Data/backend_calls.csv', delimiter=",", dtype=str) #30k

In [4]:
#make sure the hospital times are the same at different times.
hosp = np.genfromtxt('Input_Data/calls_w_StnHospTimes.csv', delimiter=",", dtype=str)
hosp

array([['', 'arrival_seconds', 'interarrival_seconds', ..., 'day',
        'minute', 'second'],
       ['51208', '0', '0', ..., '1', '7', '26'],
       ['43271', '96', '96', ..., '1', '5', '58'],
       ...,
       ['165764', '51959848', '249', ..., '24', '22', '27'],
       ['24424', '51959908', '60', ..., '24', '19', '2'],
       ['185223', '51960284', '376', ..., '24', '26', '35']], dtype='<U20')

In [5]:
hosp.shape #(210108, 67)
hosp[:,3] #neighborhood
hosp[:,5] #
where67 = np.where(hosp[:,3] == "67")
hosp[where67[0],8]

array(['1384.38', '1384.38', '1384.38', ..., '1384.38', '1384.38',
       '1384.38'], dtype='<U20')

In [6]:
backend[0]

array(['', 'Unnamed: 0', 'IncidentForeignKey', 'Radio_Name',
       'Longitude_At_Assign_Time', 'Latitude_At_Assign_Time',
       'Time_Assigned', 'Time_Enroute', 'Time_ArrivedAtScene',
       'Time_Depart_Scene', 'Time_Arrive_Destination', 'Time_Available',
       'Time_Call_Cleared', 'Call_Disposition', 'Longitude_Of_Emergency',
       'Latitude_Of_Emergency', 'transport_time', 'grid_time',
       'Time_Arrived', 'timedelta', 'arrival_seconds',
       'interarrival_seconds', 'neighborhood', 'dow'], dtype='<U30')

In [7]:
#filter 1) dow- sat and sunday out already filtered. 2) times outside of 8am-8pm
backend[0]

array(['', 'Unnamed: 0', 'IncidentForeignKey', 'Radio_Name',
       'Longitude_At_Assign_Time', 'Latitude_At_Assign_Time',
       'Time_Assigned', 'Time_Enroute', 'Time_ArrivedAtScene',
       'Time_Depart_Scene', 'Time_Arrive_Destination', 'Time_Available',
       'Time_Call_Cleared', 'Call_Disposition', 'Longitude_Of_Emergency',
       'Latitude_Of_Emergency', 'transport_time', 'grid_time',
       'Time_Arrived', 'timedelta', 'arrival_seconds',
       'interarrival_seconds', 'neighborhood', 'dow'], dtype='<U30')

In [8]:
backend[0,6] #Time_Assigned
backend[0,7] #Time_Enroute
backend[0,8] #Time_ArrivedAtScene
backend[0,9] #Time_Depart_Scene
backend[0,10] #Time_Arrive_Destination
backend[0,11] #Time_Available
backend[0,12] #Time_Call_Cleared
backend[0,16] #transport_time
backend[0,17] #grid_time

'grid_time'

In [9]:
index = 128
print(backend[index,7])
print(backend[index,8])
print(float(backend[index,16])/60)

2020-04-23 09:08:06.787
2020-04-23 09:20:14.760
12.116666666666667


## calculate times with calendar difference

In [10]:
import math
from datetime import *; from dateutil.relativedelta import *
import calendar
def csv2calendar(index,type_event):
    time1 = backend[index,type_event]
    if time1 == "":
        return -1
    [ymd,hmm] = time1.split()
    [y,m,d] = ymd.split('-')
    [h,minu,sec] = hmm.split(":")
    y = int(y)
    m = int(m)
    d = int(d)
    h = int(h)
    minu = int(minu)
    sec = math.floor(float(sec))
    timearrive = datetime(y, m, d, h, minu, sec)
    return timearrive

####  small example

In [11]:
print(backend[128,7])
a = csv2calendar(128,7)
a

2020-04-23 09:08:06.787


datetime.datetime(2020, 4, 23, 9, 8, 6)

In [12]:
print(backend[128,12])
b = csv2calendar(128,12)
b

2020-04-23 10:37:18.267


datetime.datetime(2020, 4, 23, 10, 37, 18)

In [13]:
(b-a).seconds / 60

89.2

In [25]:
backend[0,6] #Time_Assigned
backend[0,7] #Time_Enroute
backend[0,8] #Time_ArrivedAtScene
backend[0,9] #Time_Depart_Scene
backend[0,10] #Time_Arrive_Destination
backend[0,11] #Time_Available
backend[0,12] #Time_Call_Cleared
backend[0,16] #transport_time
backend[0,17] #grid_time
nsamples = 10
callAssignedTime = [] #np.zeros([10,1])
station2call = []
on_scene_time = []
call2hospital = []
hospital2station = []
end = np.zeros([10,1])

In [21]:
for i in range(1,nsamples):
    Time_Assigned = csv2calendar(i,6)
    Time_Enroute = csv2calendar(i,7)
    Time_ArrivedAtScene = csv2calendar(i,8)
    Time_Depart_Scene = csv2calendar(i,9)
    Time_Arrive_Destination = csv2calendar(i,10)
    Time_Available = csv2calendar(i,11)
    Time_Call_Cleared = csv2calendar(i,12)
    
    #callAssignedTime
    if Time_Assigned == -1 or Time_Enroute == -1:
        callAssignedTime.append(-1)
    else: 
        print(str(Time_Enroute) + "-" + str(Time_Assigned))
        callAssignedTime.append((Time_Enroute - Time_Assigned).seconds / 60)
        callAssignedTime[i] = float(t)
    
    #station2call
    if Time_ArrivedAtScene == -1 or Time_Enroute == -1:
        station2call[i] = -1
    else:    
        station2call[i] = (Time_ArrivedAtScene - Time_Enroute).seconds / 60
    
    #on_scene_time
    if Time_Depart_Scene == -1 or Time_ArrivedAtScene == -1:
        on_scene_time[i] = -1
    else:    
        on_scene_time[i] = (Time_Depart_Scene - Time_ArrivedAtScene).seconds / 60
    
    #call2hospital
    if Time_Arrive_Destination == -1 or Time_Enroute == -1:
        call2hospital[i] = -1
    else:    
        call2hospital[i] = (Time_Arrive_Destination - Time_Enroute).seconds / 60
    
    if Time_ArrivedAtScene == -1 or Time_Depart_Scene == -1:
        station2call[i] = -1
    else:    
        station2call[i] = (Time_ArrivedAtScene - Time_Depart_Scene).seconds / 60
    
    #hospital2station
    if Time_ArrivedAtScene == -1 or Time_Depart_Scene == -1:
        hospital2station[i] = -1
    else:    
        hospital2station[i] = (Time_Available - Time_Arrive_Destination).seconds / 60


2020-04-22 20:26:56-2020-04-22 20:25:53
2020-04-22 20:27:11-2020-04-22 20:25:50
2020-04-22 20:36:49-2020-04-22 20:35:32
2020-04-22 20:38:52-2020-04-22 20:37:22
2020-04-22 20:42:58-2020-04-22 20:42:13
2020-04-22 20:44:40-2020-04-22 20:43:18
2020-04-22 20:44:06-2020-04-22 20:41:49
2020-04-22 20:55:48-2020-04-22 20:55:38
2020-04-22 20:57:06-2020-04-22 20:49:45


In [24]:
print(np.mean(callAssignedTime))
print(np.mean(callAssignedTime))
print(np.mean(callAssignedTime))
print(np.mean(callAssignedTime))
print(np.mean(callAssignedTime))

1.67027027027027
1.67027027027027
1.67027027027027
1.67027027027027
1.67027027027027


In [16]:
i = 1
Time_Assigned = csv2calendar(i,6)
Time_Enroute = csv2calendar(i,7)
Time_ArrivedAtScene = csv2calendar(i,8)
Time_Depart_Scene = csv2calendar(i,9)
Time_Arrive_Destination = csv2calendar(i,10)
Time_Available = csv2calendar(i,11)
Time_Call_Cleared = csv2calendar(i,12)

#callAssignedTime
t = float((Time_Enroute - Time_Assigned).seconds / 60)
print(type(t))
t

<class 'float'>


1.05

In [ ]:
t = (Time_Enroute - Time_Assigned).seconds

In [ ]:
t + 1

In [ ]:
ls = []
ls.append(t)

In [ ]:
ls